  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/132.2 kB ? eta -:--:--
     -------------------------------------- 132.2/132.2 kB 7.6 MB/s eta 0:00:00
  Running setup.py install for bs4: started
  Running setup.py install for bs4: finished with status 'done'


  DEPRECATION: bs4 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3

     ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
     ------- -------------------------------- 0.8/3.9 MB 16.3 MB/s eta 0:00:01
     ---------------------------------------  3.8/3.9 MB 41.0 MB/s eta 0:00:01
     ---------------------------------------- 3.9/3.9 MB 35.7 MB/s eta 0:00:00


In [3]:
year = "2023"
s = Service('/chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = s) 

12

In [11]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

url = player_stats_url.format(year)
driver.get(url)
html = driver.page_source #grab the data
with open("player/{}.html".format(year),"w+",encoding="utf-8") as f:
    f.write(html)

In [12]:
dfs2 = []

with open("player/{}.html".format(year),encoding="utf-8") as f:
    page = f.read()
soup = BeautifulSoup(page,"html.parser")
tHeads = soup.findAll('tr', class_="thead")
for tHead in tHeads:
    tHead.decompose()
player_table = soup.find(id = "per_game_stats")
player = pd.read_html(str(player_table))[0]
player["Year"] = year
dfs2.append(player)
players = pd.concat(dfs2)
players

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,...,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1,2022
1,2,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,...,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9,2022
2,3,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,...,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1,2022
3,4,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,...,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1,2022
4,5,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,...,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,515,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,...,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023
647,516,Trae Young,PG,24,ATL,62,62,35.0,8.6,19.7,...,0.8,2.2,3.0,10.1,1.1,0.1,4.1,1.5,27.0,2023
648,517,Omer Yurtseven,C,24,MIA,3,0,8.3,1.0,2.0,...,0.7,1.0,1.7,0.0,0.0,0.3,0.0,2.0,2.3,2023
649,518,Cody Zeller,C,30,MIA,10,0,13.0,1.5,2.9,...,1.6,1.9,3.5,0.6,0.2,0.4,0.8,2.3,4.7,2023


In [13]:
playerscon = sqlite3.connect('players.sqlite')

In [14]:
players.to_sql("PLAYERS", con = playerscon,if_exists = 'append')

1463

In [16]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

url = team_stats_url.format(year)
driver.get(url)
html = driver.page_source #grab the data
with open("team/{}.html".format(year),"w+",encoding="utf-8") as f:
    f.write(html)

In [17]:
dfs3 = []


with open("team/{}.html".format(year),encoding="utf-8") as f:
    page = f.read()
soup = BeautifulSoup(page,"html.parser")
if(soup.find('tr', class_="thead")!=None):
     soup.find('tr', class_="thead").decompose()
team_table = soup.find(id = "divs_standings_E")
team = pd.read_html(str(team_table))[0]
team["Year"] = year
team["Team"] = team["Eastern Conference"]
del team["Eastern Conference"]
dfs3.append(team)

soup = BeautifulSoup(page,"html.parser")
if(soup.find('tr', class_="thead")!=None):

    soup.find('tr', class_="thead").decompose()

team_table = soup.find(id = "divs_standings_W")
team = pd.read_html(str(team_table))[0]
team["Year"] = year
team["Team"] = team["Western Conference"]
del team["Western Conference"]
dfs3.append(team)
teams = pd.concat(dfs3)

In [18]:
teamscon = sqlite3.connect('teams.sqlite')

In [19]:
teams.to_sql("TEAMS", con = teamscon,if_exists = 'append')

70